This notebook performs a batch process for multiple files (a range of dates/times).  Each iteration includes the following tasks:

    1. Obtain a NOAA hourly RAP Data from https://www.ncei.noaa.gov/data/rapid-refresh/access/rap-252-20km/analysis/
    2. Read the obtained GRIB2 file using pygrib Library
    3. Perform data transformation for the read file 
    4. Upload transformed data to an AWS S3 Bucket as a .csv file

Note, this notebook was executed as an AWS SageMaker Notebook Instance.

### 1. Install/Call Libraries

In [1]:
pip install pygrib

     |████████████████████████████████| 15.3 MB 12.8 MB/s eta 0:00:01
     |████████████████████████████████| 6.5 MB 62.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pygrib

import pandas as pd
import numpy as np
from datetime import datetime

import sagemaker
import boto3
import os

from urllib.request import urlopen
from shutil import copyfileobj

import math

### 2. Functions

In [3]:
# Returns level ranges based on user specified minimum & maximum altitudes
def convert_hPa (minimum_altitude, maximum_altitude):
    altitude = []
    hPa = []
    min_alt = int(minimum_altitude)
    max_alt = int(maximum_altitude)
    range_summary = []
    level_range = []
    
    # creating list of all hPa values from GRIB
    for i in range(100, 1001, 25):
        hPa.append(i)
        
    # converting hPa to pressure altitude feet using International Standard Atmosphere
    for i in hPa:
        alt = (1 - (i/1013.25)**0.190284) * 145366.45
        FL = 'FL'+str(int(round(alt/100,-1)))
        altitude.append([alt, FL, i])

    for i in altitude:
        if(i[0] >= min_alt and i[0] <= max_alt):
            range_summary.append(i)
            
    for i in range_summary:
        level_range.append(i[2])
    return(level_range)

# converts feet to pressure (hPa)
def alt_to_hPa (alt):
    a = (1/0.190284)
    b =  alt/145366.45
    P_hpa  = int(round(math.exp(math.log(1013.25) + (a*(math.log(1-b)))),0))
    return P_hpa

# Converts Relative Humidity Over Water to Realtive Humidity Over Ice
a_ice = 21.8745484
b_ice = 7.66
a_water = 17.2693882
b_water = 35.86

def convertRH_water_to_RH_ice(row):
    T = row['Temperature']
    RH_water = row['RelativeHumidity']
    RH_ice = RH_water * math.exp((a_water * (T - 273.16))/(T - b_water) - (a_ice * (T - 273.16))/(T - b_ice))
    RH_ice = round(RH_ice,2)
    return RH_ice

# Determines ISSR
def define_ISSR (row):
    if row['Temperature'] < 233.15 and row['RH_ice'] > 100:
        return 1
    else:
        return 0

# Converts pressure (hPa) to Flight Level
def hPa_to_FL (row):
    hPa = row['hPa']
    alt = (1 - (hPa/1013.25)**0.190284) * 145366.45
    #FL = 'FL'+str(int(round(alt/100,-1)))
    FL = int(round(alt/100,-1))
    return(FL) 

# Converts separate integer values of year, month, day, and hour to a single datetime value
def convertdatetime(row):
    yr = int(row['Year'])
    mo = int(row['Month'])
    dy = int(row['Day'])
    hr = int(row['Hour'])
    stamp = datetime(yr, mo, dy, hr)
    return str(stamp)

# Develop urls based on user-specified date/time range
def get_RAP_urls(firstDT, lastDT):
    url_begin = 'https://www.ncei.noaa.gov/data/rapid-refresh/access/rap-252-20km/analysis/'
    if (firstDT >= lastDT):
        lastDT = firstDT

    dateTimes = pd.date_range(firstDT, lastDT, freq= 'H')
    dateTimesSer = pd.Series([str(dateTime) for dateTime in dateTimes], name= 'temp')
    dateTimesDF = pd.DataFrame({'yr':list(dateTimesSer.str.slice(0,4)), 
                                    'mo':list(dateTimesSer.str.slice(5,7)), 
                                    'day':list(dateTimesSer.str.slice(8,10)), 
                                    'hr':list(dateTimesSer.str.slice(11,13))})

    dateTimesDF['yrmo'] = dateTimesDF['yr'] + dateTimesDF['mo']
    dateTimesDF['yrmoday'] = dateTimesDF['yrmo'] + dateTimesDF['day']
    dtDF = dateTimesDF.iloc[0:len(dateTimesDF)].copy()
    siteNames = [url_begin + dtDF.iloc[i,4] + "/" + dtDF.iloc[i,5] + '/rap_252_' + dtDF.iloc[i,5] + '_' + dtDF.iloc[i,3] + '00_000.grb2' for i in range(len(dtDF))]
    
    return siteNames

### 3. User Input for Date/Time Range 

In [9]:
# User input
beginDT = '2020-12-10 00:00:00' # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< User Input
endDT =   '2020-12-10 23:00:00' # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< User Input

urls = get_RAP_urls(beginDT, endDT)

# Print urls that the GRIB2 files will be obtained
for url in urls:
    print(url)

https://www.ncei.noaa.gov/data/rapid-refresh/access/rap-252-20km/analysis/202012/20201210/rap_252_20201210_0000_000.grb2
https://www.ncei.noaa.gov/data/rapid-refresh/access/rap-252-20km/analysis/202012/20201210/rap_252_20201210_0100_000.grb2
https://www.ncei.noaa.gov/data/rapid-refresh/access/rap-252-20km/analysis/202012/20201210/rap_252_20201210_0200_000.grb2
https://www.ncei.noaa.gov/data/rapid-refresh/access/rap-252-20km/analysis/202012/20201210/rap_252_20201210_0300_000.grb2
https://www.ncei.noaa.gov/data/rapid-refresh/access/rap-252-20km/analysis/202012/20201210/rap_252_20201210_0400_000.grb2
https://www.ncei.noaa.gov/data/rapid-refresh/access/rap-252-20km/analysis/202012/20201210/rap_252_20201210_0500_000.grb2
https://www.ncei.noaa.gov/data/rapid-refresh/access/rap-252-20km/analysis/202012/20201210/rap_252_20201210_0600_000.grb2
https://www.ncei.noaa.gov/data/rapid-refresh/access/rap-252-20km/analysis/202012/20201210/rap_252_20201210_0700_000.grb2
https://www.ncei.noaa.gov/data/r

### 4. Process Data

In [4]:
# Each iteration performs dowloading a GRIB2 file, reading the downloaded file, 
# extracting & transforming necessary data, and writing a .csv file to a user-specified AWS s3 bucket   
for my_url in urls:
    with urlopen(my_url) as in_stream, open('my_gribfile.grib2', 'wb') as out_file:
        copyfileobj(in_stream, out_file)
    grbs = pygrib.open('my_gribfile.grib2')
    
    temperature = {}
    relative_humidity = {}
    latitude = {}
    longitude = {}
    data = {}
    ISSR = {}
    latlon = {'lat_lon' : []}
    lat = []
    lon= []
    count = 0
    min_alt = int(19000)
    max_alt = int(45000)

    for grb in grbs:
            if grb.level in convert_hPa(min_alt,max_alt):
                if grb.parameterName == 'Temperature':
                    temperature.update({grb.level : {grb.parameterName : np.round(np.array(grb.values),3).tolist()}})
                    #temperature.update({grb.level : {grb.parameterName : numpy.round(numpy.array(grb.values),3), 'temp_validation' : validation()}})
                if grb.parameterName == 'Relative humidity':
                    relative_humidity.update({grb.level : {grb.parameterName : np.round(np.array(grb.values),2).tolist()}})
                    #relative_humidity.update({grb.level : {grb.parameterName : numpy.round(numpy.array(grb.values),2), 'rh_validation' : validation()}})

            lat,lon = grb.latlons()
            lat = lat.T.tolist()
            lon = lon.T.tolist()
    # adding year, month, day, hour to data dictionary
    #data = {'lat_lon' : [], 'data' : {grb.year : {grb.month : {grb.day : {grb.hour : {}}}}}} # including lat lon for each level
    data = {'data' : {grb.year : {grb.month : {grb.day : {grb.hour : {}}}}}} # excluding lat lon for each level  
    grbs.close()
    
    for year in data['data']:
            #print('data year:',year,type(year))
            for month in data['data'][year]:
                #print('data month:', month,type(month))
                for day in data['data'][year][month]:
                    #print('data day:', day,type(day))
                    for hour in data['data'][year][month][day]:
                        #print('data hour:', hour,type(hour))
                        data['data'][year][month][day][hour].update(temperature)
                        for level in data['data'][year][month][day][hour]:
                            #print('data level:', level,type(level))
                            data['data'][year][month][day][hour][level].update(relative_humidity[level])

    df = pd.DataFrame() 

        # append columns to an empty DataFrame 
    df['Year'] = [] 
    df['Month'] = [] 
    df['Day'] = [] 
    df['Hour'] = []
    df['hPa'] = []
    df['Nx'] = []
    df['Ny'] = []
    df['Lat'] = []
    df['Lon'] = []
    df['Temperature'] = []
    df['RelativeHumidity'] = []
    df['IsISSR'] = []

    counter = 0
    year_list = []
    month_list = []
    day_list = []
    hour_list = []
    level_list = []
    Nx_list = []
    Ny_list = []
    Lat_List = []
    Lon_List = []
    Temp_List = []
    RH_List = []


    for levelIndex, level in enumerate(data['data'][year][month][day][hour]):
        for variableindex, variable in enumerate(data['data'][year][month][day][hour][level]):
            if(variable == "Temperature"):
                for Ny_index, list_of_Ny_no_of_temperature_values in enumerate(data['data'][year][month][day][hour][level][variable]):
                    for Nx_index,temperature_value in enumerate(list_of_Ny_no_of_temperature_values):
                        year_list.append(year)
                        month_list.append(month)
                        day_list.append(day)
                        hour_list.append(hour)
                        level_list.append(level)
                        Nx_list.append(Nx_index+1)
                        Ny_list.append(Ny_index+1)
                        Temp_List.append(temperature_value)
                        Lat_List.append(lat[Nx_index][Ny_index])
                        Lon_List.append(lon[Nx_index][Ny_index])

            if(variable == "Relative humidity"):
                for Ny_index, list_of_Ny_no_of_temperature_values in enumerate(data['data'][year][month][day][hour][level][variable]):
                    for Nx_index,rh_value in enumerate(list_of_Ny_no_of_temperature_values):
                        RH_List.append(rh_value)
    df['Year'] = year_list 
    df['Month'] = month_list
    df['Day'] = day_list
    df['Hour'] = hour_list
    df['hPa'] = level_list
    df['Nx'] = Nx_list
    df['Ny'] = Ny_list
    df['Lat'] = Lat_List
    df['Lon'] = Lon_List
    df['Temperature'] = Temp_List
    df['RelativeHumidity'] = RH_List
    
    df['RH_ice'] = df.apply(lambda row: convertRH_water_to_RH_ice (row), axis=1)
    df['IsISSR'] = df.apply(lambda row: define_ISSR (row), axis=1)
    df['FLevel'] = df.apply(lambda row: hPa_to_FL (row), axis=1)
    df['dateTime'] = df.apply(lambda row: convertdatetime (row), axis=1)
    final_df = df[['dateTime', 'hPa', 'FLevel','Nx', 'Ny', 'Lat', 'Lon','Temperature', 'RH_ice', 'IsISSR']]
    
    stampText = final_df.iloc[0]['dateTime']
    outFileName = stampText[0:4] + "_" + stampText[5:7] + "_" + stampText[8:10] + "_" + stampText[11:13] + ".csv"
    
#    final_df.to_csv('final.csv', index= False) # local temporary storage
    
#    s3 = boto3.resource('s3')
#    s3.Bucket('partly-cloudy-XXXXXXXXXXXX').upload_file('final.csv', outFileName) # <<<<<<<<<<<<<<<<< User Input (s3 Bucket Name)
#    print(outFileName + " done at " + str(datetime.now())) # print out completed files
    

2020_12_10_00.csv done at 2021-06-09 10:09:43.638962
2020_12_10_01.csv done at 2021-06-09 10:11:40.914998
2020_12_10_02.csv done at 2021-06-09 10:13:32.565984
2020_12_10_03.csv done at 2021-06-09 10:15:26.182598
2020_12_10_04.csv done at 2021-06-09 10:17:18.418933
2020_12_10_05.csv done at 2021-06-09 10:19:10.424229
2020_12_10_06.csv done at 2021-06-09 10:21:01.233906
2020_12_10_07.csv done at 2021-06-09 10:22:52.438985
2020_12_10_08.csv done at 2021-06-09 10:24:58.858835
2020_12_10_09.csv done at 2021-06-09 10:26:54.482618
2020_12_10_10.csv done at 2021-06-09 10:28:49.858871
2020_12_10_11.csv done at 2021-06-09 10:30:41.831033
2020_12_10_12.csv done at 2021-06-09 10:32:34.658219
2020_12_10_13.csv done at 2021-06-09 10:34:28.513998
2020_12_10_14.csv done at 2021-06-09 10:36:22.506017
2020_12_10_15.csv done at 2021-06-09 10:38:14.735625
2020_12_10_16.csv done at 2021-06-09 10:40:06.966493
2020_12_10_17.csv done at 2021-06-09 10:41:59.251864
2020_12_10_18.csv done at 2021-06-09 10:43:50.